In [2]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import numpy as np
from matplotlib import pyplot as plt
import re
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk import download
import warnings
warnings.filterwarnings(action="ignore")
download('stopwords')
import re
import nltk
import time
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# Bag of words
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
import matplotlib.pyplot as plt

    # Loading advance sklearn modules
    # Sklearn Modules
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

    # Importing Models
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

    # Importing evaluation modules
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


    # Setting environment to ignore future warnings
import warnings
warnings.simplefilter('ignore')

    # Downloading & Loading NLTK modelus
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("stopwords")
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')

STOPWORDS = nltk.corpus.stopwords.words("english")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from sklearn import set_config
set_config(transform_output="pandas")


### Data Loading

In [3]:
data = pd.read_excel('/content/drive/MyDrive/Zunaira_Thesis/Zunaira/Daraz Labelled Review Dataset with Sentiments and Features.xlsx', index_col=[0])
stopwords = pd.read_csv('/content/drive/MyDrive/Zunaira_Thesis/Zunaira/roman-stop-words.csv')
df = pd.read_excel('/content/drive/MyDrive/Zunaira_Thesis/Zunaira/train.xlsx')
df = df[df['Label'] == 'fake']
df['Label'] = df['Label'].map({'fake':1})
df

,Review,Label
0,thnk amazon fori tarseel ke liye. haan yeh beh...,1
1,kitni price ka mila ap ko daba pack mojeh daba...,1
2,AOA mere vivo y19 hai agr kisi ne infinix zero...,1
3,My Husband Bought This Box Packed Phone For Me...,1
4,Kisi bhai ny sale krna ha to contact me from t...,1
...,...,...
936,mein you 12 khareedna chahta hon aur is ke ila...,1
937,Sony Xperia z3 compact sim working PTA not app...,1
938,mein htc u12 plus beech raha hon. . siyah 6 / ...,1
939,Hi every One .. plz dont deal with this guy .....,1


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from collections import Counter
Counter(df['Label'])


Counter({1: 941})

In [5]:
data.shape

(3923, 9)

In [6]:
data.columns

Index(['Product_ID', 'Customer_Name', 'Customer_ID', 'Date', 'Rating',
       'Spam(1) and Not Spam(0)', 'Reviews', 'Sentiment', 'Features'],
      dtype='object')

In [7]:
from collections import Counter
Counter(data['Spam(1) and Not Spam(0)'])


Counter({0: 3393, 1: 530})

In [8]:
data = data[['Reviews','Spam(1) and Not Spam(0)']].rename(columns={"Reviews":"Review","Spam(1) and Not Spam(0)":"Label"}).append(df).reset_index(drop=True)
data.shape

(4864, 2)

In [9]:
from sklearn.utils import resample

In [10]:
X = data['Review']
y = data['Label']

minority_indices = np.where(y == 1)[0]
majority_indices = np.where(y == 0)[0]

oversampled_minority_indices = resample(
    minority_indices,
    replace=True,
    n_samples=len(majority_indices),
    random_state=42
)

oversampled_indices = np.concatenate([majority_indices, oversampled_minority_indices])

X_oversampled = X[oversampled_indices]
y_oversampled = y[oversampled_indices]

In [ ]:
Counter(y_oversampled)

Counter({0: 3393, 1: 3393})

In [11]:
target=y_oversampled.values
data = X_oversampled.values
roman = set(stopwords['roman'])

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM,SimpleRNN,GRU

def sentiment_analysis(embedding_type,model, texts, labels):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

    if embedding_type=="Glove":
      X_train_vec =X_train
      X_test_vec=X_test
    # Choose the embedding approach (TF-IDF or CountVectorizer)
    else:
      if embedding_type == 'tfidf':
          vectorizer = TfidfVectorizer(max_features=5000)
      elif embedding_type == 'count':
          vectorizer = CountVectorizer(max_features=5000)
      X_train_vec = vectorizer.fit_transform(X_train).toarray()
      X_test_vec = vectorizer.transform(X_test).toarray()


    # Build the RNN model

    if model=="RNN":
      model = Sequential()
      model.add(Embedding(input_dim=X_train_vec.shape[1], output_dim=64, input_length=X_train_vec.shape[1]))
      model.add(SimpleRNN(12))

    elif model=="GRU":

      model = Sequential()
      model.add(Embedding(input_dim=X_train_vec.shape[1], output_dim=10, input_length=X_train_vec.shape[1]))
      model.add(GRU(12))

    elif model=="LSTM":
      model = Sequential()
      model.add(Embedding(input_dim=X_train_vec.shape[1], output_dim=10, input_length=X_train_vec.shape[1]))
      model.add(LSTM(12))


    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train_vec, y_train, epochs=1, batch_size=2048, validation_split=0.15)

    # Make predictions on the test set
    y_pred = model.predict(X_test_vec)
    y_pred = (y_pred > 0.5).astype(int)
    y_pred =np.array([i[0] for i in y_pred])

    # Print the classification report
    print("Classification Report:")
    print(classification_report(y_test, y_pred))



In [ ]:
# TF-IDF example
print("*"*10,"RNN-tfidf","*"*10)
sentiment_analysis('tfidf', "RNN",X_oversampled, y_oversampled)
print("*"*10,"GRU-tfidf","*"*10)
sentiment_analysis('tfidf', "GRU",X_oversampled, y_oversampled)
print("*"*10,"LSTM-tfidf","*"*10)
sentiment_analysis('tfidf', "LSTM",X_oversampled, y_oversampled)

In [1]:
# BOW example
print("*"*10,"RNN-BOW","*"*10)
sentiment_analysis('count', "RNN",X_oversampled, y_oversampled)
print("*"*10,"GRU-BOW","*"*10)
sentiment_analysis('count', "GRU",X_oversampled, y_oversampled)
print("*"*10,"LSTM-BOW","*"*10)
sentiment_analysis('count', "LSTM",X_oversampled, y_oversampled)


********** RNN-BOW **********


NameError: ignored

**GLOVE**

In [ ]:
def preprocess_dataset(dataset):
    cleaned_dataset = [re.sub(r"[^a-zA-Z0-9\s]", "", text) for text in dataset]
    tokenized_dataset = [word_tokenize(text) for text in cleaned_dataset]
    lowercased_dataset = [[word.lower() for word in text] for text in tokenized_dataset]

    stop_words = roman
    preprocessed_dataset = [[word for word in text if word not in stop_words] for text in lowercased_dataset]

    return preprocessed_dataset

preprocessed_dataset = preprocess_dataset(data)

In [ ]:
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

def train_and_save_glove_model(corpus, embedding_dim=200, window_size=5, output_file="glove_model.bin"):
    w2v_model = Word2Vec(corpus, vector_size=embedding_dim, window=window_size, min_count=1, workers=4)
    word_vectors = w2v_model.wv
    word_vectors.save(output_file)

train_and_save_glove_model(preprocessed_dataset)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, SimpleRNN, Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors

def train_ml_model(corpus, target, glove_model_file="glove_model.bin"):
    glove_model = KeyedVectors.load(glove_model_file)
    embeddings_index = glove_model.key_to_index
    embedding_dim = glove_model.vector_size
    embedding_matrix = np.zeros((len(embeddings_index), embedding_dim))
    word_index = {word: i for i, word in enumerate(embeddings_index)}
    for word, i in word_index.items():
        if i < len(embeddings_index):
            embedding_vector = embeddings_index[word]
            embedding_matrix[i] = embedding_vector

    max_sequence_length = max(len(sent) for sent in corpus)
    num_classes = len(set(target))

    X = np.zeros((len(corpus), max_sequence_length), dtype=np.int32)
    y = np.zeros((len(target), num_classes), dtype=np.int32)

    for i, sent in enumerate(corpus):
        for j, word in enumerate(sent):
            X[i, j] = word_index[word]
        y[i] = target[i]
    return X,y
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train,X_val,y_train[:,-1],y_val[:,-1]
# train_ml_model()
X,y = train_ml_model(preprocessed_dataset, target, glove_model_file="glove_model.bin")

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Assuming 'X_train' and 'X_test' are your training and testing sequences
def GloveEmbeddings(model):
  X_train, X_test, y_train, y_test = train_test_split(X, y_oversampled, test_size=0.2, random_state=42)
  if model=="RNN":
    # Build the model
    model = Sequential()
    # Add the Embedding layer with weights set from your GloVe embeddings
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=X_train.shape[1],
                        weights=[glove_embeddings], trainable=False))
    model.add(SimpleRNN(12))
  elif model=="GRU":
    model = Sequential()
    # Add the Embedding layer with weights set from your GloVe embeddings
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=X_train.shape[1],
                        weights=[glove_embeddings], trainable=False))
    model.add(GRU(12))

  elif model=="LSTM":
      model = Sequential()
      # Add the Embedding layer with weights set from your GloVe embeddings
      model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=X_train.shape[1],
                          weights=[glove_embeddings], trainable=False))
      model.add(LSTM(12))

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train, y_train, epochs=2, batch_size=128, validation_split=0.15)

  # Make predictions on the test set
  y_pred = model.predict(X_test)
  y_pred = (y_pred > 0.5).astype(int)
  y_pred = np.array([i[0] for i in y_pred])

  # Print the classification report
  print("Classification Report:")
  print(classification_report(y_test, y_pred))

print("*"*10,"RNN-GLOVE","*"*10)
GloveEmbeddings("RNN")
print("*"*10,"GRU-GLOVE","*"*10)
GloveEmbeddings("GRU")
print("*"*10,"LSTM-GLOVE","*"*10)
GloveEmbeddings("LSTM")


**Benefits**

1.Improvement in Performances

2.Low Bias and Low Variance

3.Robustness


